In [6]:
# apparent_motion_slow.py
# Creates an apparent-motion illusion video with two small black squares
# on a white background, flickering at ~1 Hz.

from PIL import Image, ImageDraw
import numpy as np
import imageio.v2 as imageio

# Parameters
width, height = 640, 360      # video resolution
square_size = 40              # smaller black squares
gap = 200                     # further apart (pixels)
fps = 30
duration_seconds = 2        # total video length
total_frames = int(fps * duration_seconds)

# Flicker timing (~1 Hz apparent motion)
# One full cycle (left -> blank -> right -> blank) = 1 second
left_on_frames = int(0.5 * fps)
blank_frames = int(0.5 * fps)
right_on_frames = int(0.5 * fps)
cycle_frames = left_on_frames + blank_frames + right_on_frames + blank_frames

# Compute positions (centered vertically)
center_y = height // 2
center_x = width // 2
left_center = (center_x - gap // 2, center_y)
right_center = (center_x + gap // 2, center_y)

outfile = "apparent_motion_slow.mp4"

def create_frame(show='none'):
    """Create a single video frame."""
    img = Image.new('RGB', (width, height), (255, 255, 255))  # white background
    draw = ImageDraw.Draw(img)
    if show in ('left', 'right'):
        cx, cy = left_center if show == 'left' else right_center
        x0, y0 = cx - square_size / 2, cy - square_size / 2
        x1, y1 = cx + square_size / 2, cy + square_size / 2
        draw.rectangle([x0, y0, x1, y1], fill=(0, 0, 0))
    return np.array(img)

# Write video frames
with imageio.get_writer(outfile, fps=fps, codec='libx264',
                        ffmpeg_params=['-pix_fmt', 'yuv420p']) as writer:
    for frame_idx in range(total_frames):
        cycle_pos = frame_idx % cycle_frames
        if cycle_pos < left_on_frames:
            frame = create_frame('left')
        elif cycle_pos < left_on_frames + blank_frames:
            frame = create_frame('none')
        elif cycle_pos < left_on_frames + blank_frames + right_on_frames:
            frame = create_frame('right')
        else:
            frame = create_frame('none')
        writer.append_data(frame)

print(f"Created {outfile} — {total_frames} frames @ {fps} fps ({duration_seconds:.2f}s).")


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (640, 360) to (640, 368) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Created apparent_motion_slow.mp4 — 60 frames @ 30 fps (2.00s).


Multiple -pix_fmt options specified for stream 0, only the last option '-pix_fmt yuv420p' will be used.
